In [2]:
import os

In [3]:
path_list=[]
for root,dirs,files in os.walk("C:\\Users\\Chandragupta\\Desktop\\sentiment_classification\\aclImdb"):
    for name in files:
        path = os.path.join(root,name)
        if path.endswith("txt"):
            path_list.append(path)
        

In [4]:
path_list[:10]

['C:\\Users\\Chandragupta\\Desktop\\sentiment_classification\\aclImdb\\test\\neg\\0_2.txt',
 'C:\\Users\\Chandragupta\\Desktop\\sentiment_classification\\aclImdb\\test\\neg\\10000_4.txt',
 'C:\\Users\\Chandragupta\\Desktop\\sentiment_classification\\aclImdb\\test\\neg\\10001_1.txt',
 'C:\\Users\\Chandragupta\\Desktop\\sentiment_classification\\aclImdb\\test\\neg\\10002_3.txt',
 'C:\\Users\\Chandragupta\\Desktop\\sentiment_classification\\aclImdb\\test\\neg\\10003_3.txt',
 'C:\\Users\\Chandragupta\\Desktop\\sentiment_classification\\aclImdb\\test\\neg\\10004_2.txt',
 'C:\\Users\\Chandragupta\\Desktop\\sentiment_classification\\aclImdb\\test\\neg\\10005_2.txt',
 'C:\\Users\\Chandragupta\\Desktop\\sentiment_classification\\aclImdb\\test\\neg\\10006_2.txt',
 'C:\\Users\\Chandragupta\\Desktop\\sentiment_classification\\aclImdb\\test\\neg\\10007_4.txt',
 'C:\\Users\\Chandragupta\\Desktop\\sentiment_classification\\aclImdb\\test\\neg\\10008_4.txt']

In [5]:

import random
random.shuffle(path_list)
random.shuffle(path_list)
random.shuffle(path_list)

In [6]:
print(path_list[:10])
print(len(path_list))

['C:\\Users\\Chandragupta\\Desktop\\sentiment_classification\\aclImdb\\test\\neg\\8952_1.txt', 'C:\\Users\\Chandragupta\\Desktop\\sentiment_classification\\aclImdb\\train\\pos\\2301_10.txt', 'C:\\Users\\Chandragupta\\Desktop\\sentiment_classification\\aclImdb\\train\\neg\\6040_1.txt', 'C:\\Users\\Chandragupta\\Desktop\\sentiment_classification\\aclImdb\\train\\pos\\9321_8.txt', 'C:\\Users\\Chandragupta\\Desktop\\sentiment_classification\\aclImdb\\test\\pos\\5333_10.txt', 'C:\\Users\\Chandragupta\\Desktop\\sentiment_classification\\aclImdb\\train\\neg\\8999_4.txt', 'C:\\Users\\Chandragupta\\Desktop\\sentiment_classification\\aclImdb\\test\\neg\\5821_3.txt', 'C:\\Users\\Chandragupta\\Desktop\\sentiment_classification\\aclImdb\\train\\neg\\637_3.txt', 'C:\\Users\\Chandragupta\\Desktop\\sentiment_classification\\aclImdb\\test\\pos\\7104_7.txt', 'C:\\Users\\Chandragupta\\Desktop\\sentiment_classification\\aclImdb\\test\\neg\\10504_1.txt']
50000


In [7]:
X=[]
y=[]

for p in path_list:
    myfile=open(p, encoding="utf8")
    X.append(myfile.read())
    cat=p.split('\\');
    c=(cat[7]=="pos")*1
    y.append(c)
    
    

In [8]:
len(X[0])

669

In [9]:
import tensorflow as tf
from tensorflow import keras

In [11]:
vectorize_layer = tf.keras.layers.experimental.preprocessing.TextVectorization(output_sequence_length=1000)

In [12]:
d=tf.data.Dataset.from_tensor_slices(X)
vectorize_layer.adapt(d.batch(64))

In [13]:
len(vectorize_layer.get_vocabulary())

182736

In [14]:
X_example = tf.constant(["It's a great, great movie! I loved it. you watch too", "It was terrible, run away!!!","hi"])

In [15]:
class BagOfWords(keras.layers.Layer):
    def __init__(self, n_tokens, dtype=tf.int32, **kwargs):
        super().__init__(dtype=tf.int32, **kwargs)
        self.n_tokens = n_tokens
    def call(self, inputs):
        one_hot = tf.one_hot(inputs, self.n_tokens)
        return tf.reduce_sum(one_hot, axis=1)[:, 1:]

In [16]:
bagofwords=BagOfWords(n_tokens=1002)

In [19]:
from tensorflow.keras.layers import Dense

In [17]:

model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=(1,), dtype=tf.string))
model.add(vectorize_layer)
model.add(bagofwords)
model.add(Dense(512, activation="relu"))
model.add(Dense(128, activation="relu"))
model.add(Dense(1,activation="sigmoid"))

In [18]:
model.compile(loss="binary_crossentropy", optimizer="nadam",
              metrics=["accuracy"])

In [19]:
history=model.fit(X[:25000],y[:25000],epochs=20,validation_data=(X[25000:40000],y[25000:40000]))

Train on 25000 samples, validate on 15000 samples
Epoch 1/20
25000/25000 [==============================] - 180s 7ms/sample - loss: 0.3849 - accuracy: 0.8316 - val_loss: 0.3468 - val_accuracy: 0.8481
Epoch 2/20
25000/25000 [==============================] - 168s 7ms/sample - loss: 0.2873 - accuracy: 0.8764 - val_loss: 0.3580 - val_accuracy: 0.8399
Epoch 3/20
25000/25000 [==============================] - 162s 6ms/sample - loss: 0.2112 - accuracy: 0.9131 - val_loss: 0.4741 - val_accuracy: 0.8253
Epoch 4/20
25000/25000 [==============================] - 168s 7ms/sample - loss: 0.1140 - accuracy: 0.9567 - val_loss: 0.4454 - val_accuracy: 0.8460
Epoch 5/20
25000/25000 [==============================] - 162s 6ms/sample - loss: 0.0572 - accuracy: 0.9796 - val_loss: 0.6386 - val_accuracy: 0.8421
Epoch 6/20
25000/25000 [==============================] - 164s 7ms/sample - loss: 0.0413 - accuracy: 0.9858 - val_loss: 0.7618 - val_accuracy: 0.8464
Epoch 7/20
25000/25000 [==========================

In [20]:
model.evaluate(X[40000:],y[40000:])

10000/10000 [==============================] - 39s 4ms/sample - loss: 1.0078 - accuracy: 0.8491


[1.0077831466689706, 0.8491]

In [ ]:
len(vectorize_layer.get_vocabulary())

In [47]:
X_example=["good","bad","not bad, good","not boring","boring"]

In [48]:
p=model.predict(X_example)
for i in p:
    if i>.5:
        print("positive")
    else:
        print("negative")

positive
negative
negative
negative
negative


In [30]:
X[0]

'When I first saw a glimpse of this movie, I quickly noticed the actress who was playing the role of Lucille Ball. Rachel York\'s portrayal of Lucy is absolutely awful. Lucille Ball was an astounding comedian with incredible talent. To think about a legend like Lucille Ball being portrayed the way she was in the movie is horrendous. I cannot believe out of all the actresses in the world who could play a much better Lucy, the producers decided to get Rachel York. She might be a good actress in other roles but to play the role of Lucille Ball is tough. It is pretty hard to find someone who could resemble Lucille Ball, but they could at least find someone a bit similar in looks and talent. If you noticed York\'s portrayal of Lucy in episodes of I Love Lucy like the chocolate factory or vitavetavegamin, nothing is similar in any way-her expression, voice, or movement.<br /><br />To top it all off, Danny Pino playing Desi Arnaz is horrible. Pino does not qualify to play as Ricky. He\'s smal

In [31]:
y[0]

0

In [32]:
path_list[0]

'C:\\Users\\Chandragupta\\Desktop\\sentiment_classification\\aclImdb\\train\\neg\\120_1.txt'